In [24]:
import os
import csv
import pandas as pd
from PIL import Image
from io import BytesIO
from fastparquet import ParquetFile, write

root = "./"
raw = os.path.join(root, "raw")
descriptions = os.path.join(root, 'short_descriptions', 'descriptions.csv')

pokemon_list = [name for name in os.listdir(raw)
    if os.path.isdir(os.path.join(raw, name))]

dataset = {'text': [], 'image': []}

def convertToJpeg(im):
    with BytesIO() as f:
        im.save(f, format='JPEG', quality=100, subsampling=0)
        return f.getvalue()

with open(descriptions, newline='', encoding="utf-8") as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
        name = row['name']
        description = row['short_description']
        img_path = os.path.join(root, "front", name + ".png")
        
        if os.path.exists(img_path) and description != None and description != '':
            png_image = Image.open(img_path).convert('RGB')

            image_bytes = convertToJpeg(png_image)
            dataset['text'].append(description)
            dataset['image'].append({'bytes': image_bytes, 'path': None})


sprites_df = pd.DataFrame.from_dict(dataset)
sprites_df.to_parquet('sprites.parquet')
# print(sprites_df)
